In [3]:
#parameters
experimental_datafile = 'inputs/results.csv'
domain_configfile = 'inputs/borrowing_hydrogen_domain.json'
batch_size = 10

In [10]:
# from summit.strategies import TSEMO
from summit.models import GPyModel

from gp_optimizer import DirectOpt
# from summit.optimizers import EnumerationOptimizer
# from summit.objective import HV
# from summit.domain import *
import numpy as np
import matplotlib.pyplot as plt

In [3]:
solvent_ds.head(5)

,stenutz_name,cosmo_name,chemical_formula,molecular_weight,density,molar_volume,refractive_index,molecular_refractive_power,dielectric_constant,dipole_moment,...,boiling_point,viscosity,partition_coefficient,vapour_pressure,henry_constant,sigma_1,sigma_2,sigma_3,sigma_4,sigma_5
cas_number,,,,,,,,,,,,,,,,,,,,,
98-07-7,(trichloromethyl)benzene,(trichloromethyl)-benzene,C7H5Cl3,195,1.173,166.6,1.557,53.67,6.9,2.14,...,220,1.85,3.68,0.2,2015.87,0.2054,18.7984,0,0,3.2973
111-78-4,"(1Z,5Z)-cycloocta-1,5-diene","1,5-cyclooctadiene",C8H12,108,0.88,122.9,1.493,35.73,2.38,0.17,...,148,1,2.98,8.87,1884.53,0,15.2165,0.4711,0,0.9742
5194-51-4,"(2E,4E)-2,4-hexadiene","trans,trans-2,4-hexadiene",C6H10,82,0.727,112.9,1.456,30.7,2.22,0.36,...,79,0.37,3.05,225.18,2089.16,0,14.9023,0.0451,0,1.3634
98-87-3,(dichloromethyl)benzene,(dichloromethyl)-benzene,C7H6Cl2,161,1.25,128.8,1.55,41.04,6.9,2.05,...,205,1.99,2.94,0.24,2235.31,0.8684,16.7703,0,0.0081,3.8876
2004-70-8,"(E)-1,3-pentadiene",1-trans-3-pentadiene,C5H8,68,0.683,99.7,1.43,25.77,2.32,0.68,...,42,0.3,2.47,702.08,2326.13,0,12.8734,0.0397,0,1.8346


In [4]:
#Read in solubility data
solubilities = pd.read_csv('inputs/solubilities.csv')
solubilities = solubilities.set_index('cas_number')
solubilities = DataSet.from_df(solubilities)
solubilities.head(5)

,solubility
cas_number,
98-07-7,-5.348250
111-78-4,-6.310797
5194-51-4,-6.592425
98-87-3,-0.808447
2004-70-8,-6.135296


In [5]:
ucb_ds.head(5)

,solvent_class,solvent_name
cas_number,,
76-05-1,Acid,Trifluoroacetic acid (TFA)
108-24-7,Acid,Acetic anhydride (Ac2O)
1493-13-6,Acid,Triflic acid
50-21-5,Acid,Lactic Acid
64-18-6,Acid,Formic acid


In [6]:
#Merge data sets
solvent_ds_full = solvent_ds.join(solubilities)
solvent_ds_final = pd.merge(solvent_ds_full, ucb_ds, left_index=True,right_index=True)
print(f"{solvent_ds_final.shape[0]} solvents for optimization")
solvent_ds_final.head(5) #Show first 5 rows of dataset

80 solvents for optimization


,stenutz_name,cosmo_name,chemical_formula,molecular_weight,density,molar_volume,refractive_index,molecular_refractive_power,dielectric_constant,dipole_moment,...,vapour_pressure,henry_constant,sigma_1,sigma_2,sigma_3,sigma_4,sigma_5,solubility,solvent_class,solvent_name
cas_number,,,,,,,,,,,,,,,,,,,,,
56-81-5,"1,2,3-propanetriol",glycerol,C3H8O3,92,1.26,73.1,1.475,20.56,47,2.56,...,0.01,3779.8,1.392,8.3081,2.465,0.5226,3.8766,0.000000,Alcohol,Glycerol
107-06-2,"1,2-dichloroethane","1,2-dichloroethane",C2H4Cl2,99,1.253,79,1.445,21.01,10.36,1.83,...,222.09,2904.4,1.2021,10.4996,0,0,3.5413,-2.004364,Halogenated,"1,2-Dichloroethane (DCE)"
110-71-4,"1,2-dimethoxyethane","1,2-dimethoxyethane",C4H10O2,90,0.869,103.7,1.38,24,7.2,1.71,...,83.42,2260.69,0,12.4399,1.8467,0,1.8566,-2.891152,Ether,Dimethoxyethane (DME)
57-55-6,"1,2-propanediol",propyleneglycol,C3H8O2,76,1.036,73.4,1.431,19.01,32,2.27,...,0.22,1564.35,1.1692,8.5231,1.9315,0.5211,2.2691,0.000000,Alcohol,"1,2-Propanediol"
7226-23-5,"1,3-dimethyltetrahydropyrimidin-2(1H)-one","1,3-dimethyltetrahydropyrimidin-2(1h)-one",C6H12N2O,128,1.064,120.5,1.488,34.71,36.12,4.17,...,0.16,5535.65,0,14.9718,1.6577,0,2.1763,-1.716153,Dipolar aprotic,Dimethyl phosphonyl urea (DMPU)


In [7]:
#Double check that there are no NaNs in the descriptors
values = solvent_ds_final.descriptors_to_numpy()
values = values.astype(np.float64)
check = np.isnan(values)
np.where(check==True)

(array([], dtype=int64), array([], dtype=int64))

## 2. Design Space

We now need to setup the design space, which is just the descriptors for the solvents.

In [8]:
domain = None
domain = Domain()
domain += DescriptorsVariable(name='solvent',
                             description='solvent for the borrowing hydrogen reaction',
                             ds=solvent_ds_final)
domain

Name,Type,Description,Values
solvent,descriptors,solvent for the borrowing hydrogen reaction,80 examples of 19 descriptors


In [ ]:
#Import data
# domain = Domain.from_configfile(domain_configfile)
previous_results = DataSet.from_csv(experimental_datafile)

In [ ]:
#Run the optimization
kernel = GPy.kern.Matern52(input_dim = domain.num_continuous_dimension+domain.num_discrete_variables, ARD=TRUE)
models = [GPyModel(kernel=kernel) for _ in range(2)]
tsemo = TSEMO(domain, models, acquisition=HV(), optimizer=EnumerationOptimizer())
design = tsemo.generate_experiments(previous_results, batch_size)

In [ ]:
#Generate some nice plots

##3d plots of the GP models and original data in 2D pc space

##Plots of the models' hyperparameters for each batch including this one

##Plots of the models' log likelihood after each batch including this one

##Plots of the hypervolume for each batch inlcuding this one

In [ ]:
#Save design for next experiment(s) and model hyperparameters to disk                                     
next_batch_num = previous_results['batch_number'].max() + 1
design.insert(0, 'batch_number', np.ones(design.shape[0])*next_batch_num, type='METADATA')
design.to_csv(f'outputs/batch_{next_batch_num}.csv')
tsemo.save_params(f'outputs/{batch}')